In [1]:
%load_ext autoreload
%autoreload 2

# Unbiased ECMWF

Here we propose a small model which is a debiased ECMWF forecast according to the data we have.
The plan is
* Compute the bias between the ECMWF model and the observations
* Make a debiased model
* Turn this model into a probabilistic forecast
For this notebook we want to do it on precipitation and temperature, for weeks 1-2, 3-4, and 5-6.

In [2]:
import dask
import dask.array as da
import dask.distributed
import datetime
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import pathlib
import scipy.stats
import typing
import xarray as xr
import xskillscore as xs

In [3]:
from crims2s.dask import create_dask_cluster
from crims2s.util import fix_dataset_dims

In [ ]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
OBSERVATIONS = '***BASEDIR***/processed/training-output-reference/'
BENCHNMARK = '***BASEDIR***training-output-benchmark/'

## Boost dask cluster

In [ ]:
cluster = create_dask_cluster()
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

## Generic Functions

In [ ]:
def extract_train_validation_from_lead_time(xr_data) -> typing.Tuple:
    xr_data_sub_train = xr_data.sel(forecast_year=slice(None, 2018))
    xr_data_sub_val = xr_data.sel(forecast_year=slice(2019, None))
    
    return xr_data_sub_train, xr_data_sub_val

In [ ]:
def compute_and_correct_bias(data_center_train, data_center_val, obs_train):
    
    bias = (obs_train - data_center_train).mean(dim=['lead_time', 'forecast_year'])
    corrected_bias = data_center_val + bias
    
    return bias, corrected_bias

In [ ]:
def add_biweekly_dim(dataset):
    weeklys = []
    for s in [slice('0D', '13D'), slice('14D', '27D'), slice('28D', '41D')]:
        weekly_forecast = dataset.sel(lead_time=s)

        first_lead = pd.to_timedelta(weekly_forecast.lead_time[0].item())

        weekly_forecast = weekly_forecast.expand_dims(dim='biweekly_forecast').assign_coords(biweekly_forecast=[first_lead])
        weekly_forecast = weekly_forecast.assign_coords(lead_time=(weekly_forecast.lead_time - first_lead))
        weeklys.append(weekly_forecast)
        
    return xr.concat(weeklys, dim='biweekly_forecast').transpose('forecast_year', 'forecast_dayofyear', 'biweekly_forecast', ...)

## Read data

### ECMWF Temperature

In [ ]:
CENTER = 'ecmwf'
FIELD = 'tp'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
input_files_tp = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
input_files_tp[:10]

In [ ]:
ecmwf_tp_raw = xr.open_mfdataset(input_files_tp, preprocess=fix_dataset_dims)
ecmwf_tp_raw = ecmwf_tp_raw.assign_coords(lead_time=ecmwf_tp_raw.lead_time - ecmwf_tp_raw.lead_time[0])
# Fix the lead times by starting them at 0. To be validated with the organizers.

In [ ]:
ecmwf_tp = add_biweekly_dim(ecmwf_tp_raw)

In [ ]:
ecmwf_tp

### Observations

In [ ]:
obs_path = pathlib.Path(OBSERVATIONS)
obs_files = [f for f in obs_path.iterdir() if 'tp' in f.stem]

In [ ]:
obs_files[:4]

In [ ]:
obs_tp_raw = xr.open_mfdataset(obs_files)
obs_tp_raw = obs_tp_raw.assign_coords(lead_time=obs_tp_raw.lead_time - obs_tp_raw.lead_time[0])

In [ ]:
obs_tp = add_biweekly_dim(obs_tp_raw)

In [ ]:
obs_tp

For precipitation we first have to take the biweekly total precip. We can't compute the difference directly on the daily forecasts.

In [ ]:
ecmwf_tp = ecmwf_tp.isel(lead_time=-1) - ecmwf_tp.isel(lead_time=0)

In [ ]:
ecmwf_tp.isel(biweekly_forecast=1, forecast_dayofyear=0, forecast_year=0, realization=0).compute().tp.plot()

In [ ]:
obs_tp = obs_tp.isel(lead_time=-1) - obs_tp.isel(lead_time=0)

In [ ]:
ecmwf_tp_train, ecmwf_tp_val = extract_train_validation_from_lead_time(ecmwf_tp)

In [ ]:
obs_tp_train, obs_tp_val = extract_train_validation_from_lead_time(obs_tp)

In [ ]:
ecmwf_tp_train

In [ ]:
obs_tp_train

## Debiasing

In [ ]:
ecmwf_tp_train

### Compute bias using training data

In [ ]:
ecmwf_tp_bias = (obs_tp_train - ecmwf_tp_train).mean(dim=['forecast_year'])

In [ ]:
ecmwf_tp_bias

### Bias correct ECMWF

In [ ]:
ecmwf_tp_val_corrected = ecmwf_tp_val + ecmwf_tp_bias

In [ ]:
ecmwf_tp_val_corrected

In [ ]:
ecmwf_tp_val_corrected_comp = ecmwf_tp_val_corrected.compute()

## Turn into probabilistic forecast

### Get thresholds from train observations

In [ ]:
obs_tp_train_thresholds = obs_tp_train.chunk({'forecast_year': -1}).quantile([0.33, 0.67], dim=['forecast_year'])

In [ ]:
obs_tp_train_thresholds

In [ ]:
obs_tp_train_thresholds_comp = obs_tp_train_thresholds.compute()

### Compute p of thresholds according to the model

There are two ways to do this. 
We can either count the amount of members that are whithin each category.
Or compute a distribution of all the members of the model, and then compute the value of the CDF for each threshold.

Here we do it using the distribution method.

#### Compute a distribution of the members of the model

In [ ]:
ecmwf_tp_val_corrected_mean = ecmwf_tp_val_corrected_comp.mean(dim=['realization'])
ecmwf_tp_val_corrected_std = ecmwf_tp_val_corrected_comp.std(dim=['realization'])

#### Compute the value of the CDF for each threshold

In [ ]:
ecmwf_tp_val_corrected_mean

In [ ]:
ecmwf_tp_val_corrected_mean.isel(biweekly_forecast=1, forecast_dayofyear=25).tp.plot()

In [ ]:
obs_tp_train_thresholds_comp.isel(biweekly_forecast=2, quantile=0, forecast_dayofyear=40).tp.plot()

In [ ]:
def make_probabilistic(forecast, thresholds):   
    loc = forecast.mean(dim=['realization']).compute().tp
    scale = forecast.std(dim=['realization']).compute().tp
    
    cdfs = xr.apply_ufunc(scipy.stats.norm.cdf, thresholds.tp, dask='allowed', kwargs={'loc': loc, 'scale': scale})
    
    below = cdfs.isel(quantile=0).drop_vars('quantile')
    normal = (cdfs.isel(quantile=1) - cdfs.isel(quantile=0))
    above = xr.ones_like(normal) - cdfs.isel(quantile=1).drop_vars('quantile')
    
    return xr.Dataset({'tp': xr.concat([below, normal, above], 'category').assign_coords(category=['below normal', 'near normal', 'above normal'])})

In [ ]:
val_probabilistic_forecast = make_probabilistic(ecmwf_tp_val_corrected_comp, obs_tp_train_thresholds_comp)

In [ ]:
val_probabilistic_forecast = val_probabilistic_forecast.expand_dims('forecast_year').assign_coords(forecast_year=ecmwf_tp_val_corrected_comp.forecast_year)

In [ ]:
#val_probabilistic_forecast = val_probabilistic_forecast.assign_coords(valid_time=ecmwf_t2m_val_corrected_comp.valid_time)

In [ ]:
val_probabilistic_forecast.biweekly_forecast.data

In [ ]:
val_probabilistic_forecast

In [ ]:
val_probabilistic_forecast = val_probabilistic_forecast.rename_dims({'biweekly_forecast': 'lead_time'}).assign_coords(lead_time=val_probabilistic_forecast.biweekly_forecast.data)

In [ ]:
val_probabilistic_forecast

In [ ]:
val_probabilistic_forecast.to_netcdf('***BASEDIR***/test_tp_forecast.nc')

In [ ]:
val_probabilistic_forecast.isel(category=2, forecast_dayofyear=40, lead_time=1).tp.plot()

In [ ]:
val_probabilistic_forecast.isel(category=1, forecast_dayofyear=40, biweekly_forecast=0).plot()

### Sanity check

In [ ]:
val_probabilistic_forecast.sum(dim='category').isel(forecast_dayofyear=0, lead_time=2).tp.plot()

## Make submission file out of it

In [ ]:
val_probabilistic_forecast_unfixed = val_probabilistic_forecast.stack(forecast_time=['forecast_year', 'forecast_dayofyear'])

In [ ]:
val_probabilistic_forecast_unfixed

In [ ]:
forecast_times = []
for f in val_probabilistic_forecast_unfixed.forecast_time:
    year, dayofyear = f.data.item()
    year = pd.to_datetime(f'{year}-01-01')
    dayofyear = pd.Timedelta(dayofyear - 1, 'D')
    forecast_times.append(year + dayofyear)

In [ ]:
forecast_time = xr.DataArray(forecast_times, dims='forecast_time')
val_probabilistic_forecast_unfixed.assign_coords(forecast_time=forecast_time).to_netcdf('***BASEDIR***/test_tp_forecast.nc')